In [1]:
import requests
import json
import pandas as pd
from arcgis import env
import os


In [1]:
!pip list

Package                           Version
--------------------------------- -----------------
anyio                             3.5.0
appdirs                           1.4.4
arcgis                            2.2.0.1
argon2-cffi                       21.3.0
argon2-cffi-bindings              21.2.0
asttokens                         2.0.5
attrs                             22.1.0
azure-core                        1.12.0
azure-storage-blob                12.8.0
Babel                             2.11.0
backcall                          0.2.0
beautifulsoup4                    4.12.2
black                             0.0.0
bleach                            4.1.0
blinker                           1.4
Bottleneck                        1.3.5
brotlipy                          0.7.0
cachetools                        4.2.2
certifi                           2023.7.22
cffi                              1.15.1
cftime                            1.6.2
chardet                           4.0.0
charset-normal

In [ ]:
#DO NOT RUN THIS CELL
#This cell only use for state FIPs code reference
state[["state","NAME"]].sort_values("NAME")


,state,NAME
46,01,Alabama
47,02,Alaska
48,04,Arizona
49,05,Arkansas
50,06,California
51,08,Colorado
0,09,Connecticut
1,10,Delaware
2,11,District of Columbia
3,12,Florida


In [31]:
#CREATING THE FUNCTIONS

def create_header(df):
  #Using first row of the dataframe to be the headers
  df.columns = df.iloc[0].values

  #Drop first row -> it's a header now
  df.drop(index = 0, inplace = True)

  #Reset index for future use
  df.reset_index(inplace = True)
  return df

def create_sum_col(df,new_col):
  #Isolating the variables columns
  cols_to_convert = df.loc[:,~df.columns.isin(["index", "NAME", "state", "county_name", "county", "tract", "block"])].columns

  #Convert the variables to integer to sum them up
  df[cols_to_convert] = df[cols_to_convert].astype(int)
  df[new_col] = df[cols_to_convert].sum(axis =1 )

def merge_df(df1,df2):
  #Assign the variable columns to the last columns of the totalpop df
  df1[df2.columns[-1]] = df2[df2.columns[-1]]
  return df1

In [32]:
#OBTAINING DATA AT BLOCK LEVEL
#Read APIs and turn into dataframe for preprocessing
variable= "P1_001N"
state_num = str(input("What is the state number you want to obtain data from? "))
totalpop = pd.read_json(f"https://api.census.gov/data/2020/dec/dhc?get=P1_001N,NAME&for=block:*&in=state:{state_num}&in=county:*&in=tract:*")
aund1 = pd.read_json(f"https://api.census.gov/data/2020/dec/dhc?get=P14_003N,P14_028N,NAME&for=block:*&in=state:{state_num}&in=county:*&in=tract:*")
a1to4 = pd.read_json(f"https://api.census.gov/data/2020/dec/dhc?get=P14_004N,P14_005N,P14_006N,P14_007N,P14_025N,P14_026N,P14_027N,P14_028N,NAME&for=block:*&in=state:{state_num}&in=county:*&in=tract:*")
a5to17 = pd.read_json(f"https://api.census.gov/data/2020/dec/dhc?get=P14_008N,P14_009N,P14_010N,P14_011N,P14_012N,P14_013N,P14_014N,P14_015N,P14_016N,P14_017N,P14_018N,P14_019N,P14_020N,P14_029N,P14_030N,P14_031N,P14_032N,P14_033N,P14_034N,P14_035N,P14_036N,P14_037N,P14_038N,P14_039N,P14_040N,P14_041N,NAME&for=block:*&in=state:{state_num}&in=county:*&in=tract:*")
a18to24 = pd.read_json(f"https://api.census.gov/data/2020/dec/dhc?get=P12_007N,P12_008N,P12_009N,P12_010N,P12_031N,P12_032N,P12_033N,P12_034N,NAME&for=block:*&in=state:{state_num}&in=county:*&in=tract:*")
aund25 = pd.read_json(f"https://api.census.gov/data/2020/dec/dhc?get=P12_007N,P12_008N,P12_009N,P12_010N,P12_031N,P12_032N,P12_033N,P12_034N,P14_003N,P14_004N,P14_005N,P14_006N,P14_007N,P14_008N,P14_009N,P14_010N,P14_011N,P14_012N,P14_013N,P14_014N,P14_015N,P14_016N,P14_017N,P14_018N,P14_019N,P14_020N,P14_024N,P14_025N,P14_026N,P14_027N,P14_028N,P14_029N,P14_030N,P14_031N,P14_032N,P14_033N,P14_034N,P14_035N,P14_036N,P14_037N,P14_038N,P14_039N,P14_040N,P14_041N,NAME&for=block:*&in=state:{state_num}&in=county:*&in=tract:*")
a25ov = pd.read_json(f"https://api.census.gov/data/2020/dec/dhc?get=P12_011N,P12_012N,P12_013N,P12_014N,P12_015N,P12_016N,P12_017N,P12_018N,P12_019N,P12_020N,P12_021N,P12_022N,P12_023N,P12_024N,P12_025N,P12_035N,P12_036N,P12_037N,P12_038N,P12_039N,P12_040N,P12_041N,P12_042N,P12_043N,P12_044N,P12_045N,NAME&for=block:*&in=state:{state_num}&in=county:*&in=tract:*")
a25to64 = pd.read_json(f"https://api.census.gov/data/2020/dec/dhc?get=P12_011N,P12_012N,P12_013N,P12_014N,P12_015N,P12_016N,P12_017N,P12_018N,P12_019N,P12_035N,P12_036N,P12_037N,P12_038N,P12_039N,P12_040N,P12_041N,P12_042N,P12_043N,NAME&for=block:*&in=state:{state_num}&in=county:*&in=tract:*")
a65to79 = pd.read_json(f"https://api.census.gov/data/2020/dec/dhc?get=P12_020N,P12_021N,P12_022N,P12_023N,P12_035N,P12_036N,P12_037N,P12_038N,P12_039N,P12_040N,P12_041N,P12_042N,P12_043N,NAME&for=block:*&in=state:{state_num}&in=county:*&in=tract:*")
a80ov = pd.read_json(f"https://api.census.gov/data/2020/dec/dhc?get=P12_024N,P12_025N,P12_048N,P12_049N,NAME&for=block:*&in=state:{state_num}&in=county:*&in=tract:*")
womchild = pd.read_json(f"https://api.census.gov/data/2020/dec/dhc?get=P12_030N,P12_031N,P12_032N,P12_033N,P12_034N,P12_035N,P12_036N,P12_037N,P12_038N,P12_039N,NAME&for=block:*&in=state:{state_num}&in=county:*&in=tract:*")
white = pd.read_json(f"https://api.census.gov/data/2020/dec/dhc?get=P3_002N,NAME&for=block:*&in=state:{state_num}&in=county:*&in=tract:*")
black = pd.read_json(f"https://api.census.gov/data/2020/dec/dhc?get=P3_003N,NAME&for=block:*&in=state:{state_num}&in=county:*&in=tract:*")
amind = pd.read_json(f"https://api.census.gov/data/2020/dec/dhc?get=P3_004N,NAME&for=block:*&in=state:{state_num}&in=county:*&in=tract:*")
asian = pd.read_json(f"https://api.census.gov/data/2020/dec/dhc?get=P3_005N,NAME&for=block:*&in=state:{state_num}&in=county:*&in=tract:*")
hawpi = pd.read_json(f"https://api.census.gov/data/2020/dec/dhc?get=P3_006N,NAME&for=block:*&in=state:{state_num}&in=county:*&in=tract:*")
other = pd.read_json(f"https://api.census.gov/data/2020/dec/dhc?get=P3_007N,NAME&for=block:*&in=state:{state_num}&in=county:*&in=tract:*")
twomore = pd.read_json(f"https://api.census.gov/data/2020/dec/dhc?get=P3_008N,NAME&for=block:*&in=state:{state_num}&in=county:*&in=tract:*")
nhisp = pd.read_json(f"https://api.census.gov/data/2020/dec/dhc?get=P4_002N,NAME&for=block:*&in=state:{state_num}&in=county:*&in=tract:*")
hisp = pd.read_json(f"https://api.census.gov/data/2020/dec/dhc?get=P4_003N,NAME&for=block:*&in=state:{state_num}&in=county:*&in=tract:*")
nhispwhite = pd.read_json(f"https://api.census.gov/data/2020/dec/dhc?get=P9_005N,NAME&for=block:*&in=state:{state_num}&in=county:*&in=tract:*")
nhispblack = pd.read_json(f"https://api.census.gov/data/2020/dec/dhc?get=P9_006N,NAME&for=block:*&in=state:{state_num}&in=county:*&in=tract:*")
household = pd.read_json(f"https://api.census.gov/data/2020/dec/dhc?get=P16_001N,NAME&for=block:*&in=state:{state_num}&in=county:*&in=tract:*")
hhpop = pd.read_json(f"https://api.census.gov/data/2020/dec/dhc?get=H8_001N,NAME&for=block:*&in=state:{state_num}&in=county:*&in=tract:*")
femhhw18c = pd.read_json(f"https://api.census.gov/data/2020/dec/dhc?get=P12_030N,P12_031N,P12_032N,P12_033N,P12_034N,P12_035N,P12_036N,P12_037N,P12_038N,P12_039N,NAME&for=block:*&in=state:{state_num}&in=county:*&in=tract:*")
hhunit = pd.read_json(f"https://api.census.gov/data/2020/dec/dhc?get=H1_001N,NAME&for=block:*&in=state:{state_num}&in=county:*&in=tract:*")
occhunit = pd.read_json(f"https://api.census.gov/data/2020/dec/dhc?get=H3_001N,NAME&for=block:*&in=state:{state_num}&in=county:*&in=tract:*")
ownerocc = pd.read_json(f"https://api.census.gov/data/2020/dec/dhc?get=H10_001N,NAME&for=block:*&in=state:{state_num}&in=county:*&in=tract:*")
vacant = pd.read_json(f"https://api.census.gov/data/2020/dec/dhc?get=H5_001N,NAME&for=block:*&in=state:{state_num}&in=county:*&in=tract:*")
hu1p = pd.read_json(f"https://api.census.gov/data/2020/dec/dhc?get=H12_003N,H12_011N,NAME&for=block:*&in=state:{state_num}&in=county:*&in=tract:*")
#Using this to reference back the state number
state = pd.read_json(f"https://api.census.gov/data/2020/dec/dhc?get={variable},NAME&for=state:*")
state = state[1:]
state_name = totalpop[1][1].split(",")[-1]
print(f"The state you are obtaining data from is{state_name}")

#Using regex to delete the redundant 0 in front of the "county" column
# block['county'] = block['county'].str.replace(r'^(0+)(?!$)', '', regex=True).fillna('0')

What is the state number you want to obtain data from? 02
The state you are obtaining data from is Alaska


In [8]:
create_header(totalpop)
create_header(aund1)
create_header(a1to4)
create_header(a5to17)
create_header(a18to24)
create_header(aund25)
create_header(a25ov)
create_header(a25to64)
create_header(a65to79)
create_header(a80ov)
create_header(womchild)
create_header(white)
create_header(black)
create_header(amind)
create_header(asian)
create_header(hawpi)
create_header(other)
create_header(twomore)
create_header(nhisp)
create_header(hisp)
create_header(nhispwhite)
create_header(nhispblack)
create_header(household)
create_header(hhpop)
create_header(femhhw18c)
create_header(hhunit)
create_header(occhunit)
create_header(ownerocc)
create_header(vacant)
create_header(hu1p)
create_sum_col(totalpop, "TOTALPOP")
create_sum_col(aund1, "AUND1")
create_sum_col(a1to4, "A1TO4")
create_sum_col(a5to17, "A5TO17")
create_sum_col(a18to24, "A18TO24")
create_sum_col(aund25, "AUND25")
create_sum_col(a25ov, "A25OV")
create_sum_col(a25to64, "A25TO64")
create_sum_col(a65to79, "A65TO79")
create_sum_col(a80ov, "A80OV")
create_sum_col(womchild, "WOMCHILD")
create_sum_col(white, "WHITE")
create_sum_col(black, "BlACK")
create_sum_col(amind, "AMIND")
create_sum_col(asian, "ASIAN")
create_sum_col(hawpi, "HAWPI")
create_sum_col(other, "OTHER")
create_sum_col(twomore, "TWOMORE")
create_sum_col(nhisp, "NHISP")
create_sum_col(hisp, "HISP")
create_sum_col(nhispwhite, "NHISPWHITE")
create_sum_col(nhispblack, "NHISPBLACK")
create_sum_col(household, "HOUSEHOLD")
create_sum_col(hhpop, "HHPOP")
create_sum_col(femhhw18c, "FEMW18CHILD")
create_sum_col(hhunit, "HHUNIT")
create_sum_col(occhunit, "OCC")
create_sum_col(ownerocc, "OWN")
create_sum_col(vacant, "VACANT")
create_sum_col(hu1p, "HU1P")
merge_df(totalpop, aund1)
merge_df(totalpop, a1to4)
merge_df(totalpop, a5to17)
merge_df(totalpop, a18to24)
merge_df(totalpop, aund25)
merge_df(totalpop, a25ov)
merge_df(totalpop, a25to64)
merge_df(totalpop, a65to79)
merge_df(totalpop, a80ov)
merge_df(totalpop, womchild)
merge_df(totalpop, white)
merge_df(totalpop, black)
merge_df(totalpop, amind)
merge_df(totalpop, asian)
merge_df(totalpop, hawpi)
merge_df(totalpop, other)
merge_df(totalpop, twomore)
merge_df(totalpop, nhisp)
merge_df(totalpop, hisp)
merge_df(totalpop, nhispwhite)
merge_df(totalpop, nhispblack)
merge_df(totalpop, household)
merge_df(totalpop, hhpop)
merge_df(totalpop, femhhw18c)
merge_df(totalpop, hhunit)
merge_df(totalpop, occhunit)
merge_df(totalpop, ownerocc)
merge_df(totalpop, vacant)
merge_df(totalpop, hu1p)
print(f"Finished processing for{state_name}")

Finished processing for Virginia


In [9]:
totalpop = totalpop.iloc[:,3:]
totalpop.insert(loc = 0, column = "GEOID", value = totalpop["state"] + totalpop["county"] + totalpop["tract"] + totalpop["block"])
totalpop.head()

,GEOID,state,county,tract,block,TOTALPOP,AUND1,A1TO4,A5TO17,A18TO24,...,NHISPWHITE,NHISPBLACK,HOUSEHOLD,HHPOP,FEMW18CHILD,HHUNIT,OCC,OWN,VACANT,HU1P
0,510010907004016,51,001,090700,4016,7,0,1,2,0,...,3,0,0,7,1,5,5,0,5,0
1,510010907004013,51,001,090700,4013,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,510010907004014,51,001,090700,4014,87,0,2,10,2,...,75,3,50,87,19,65,65,50,15,15
3,510010907004015,51,001,090700,4015,12,0,0,2,0,...,8,1,7,12,7,7,7,7,0,3
4,510010907004017,51,001,090700,4017,24,0,0,0,2,...,23,0,11,24,8,16,16,11,5,8


In [10]:
import os
folder_path = "G:/usgrids2020/processed_census_data"
os.makedirs(folder_path, exist_ok = True)
totalpop.to_csv(os.path.join(folder_path,f"{state_name.lower()}_us_final.csv"))

In [13]:
for state_num in range(53,57):
    state_num = str(state_num)
    variable= "P1_001N"
    totalpop = pd.read_json(f"https://api.census.gov/data/2020/dec/dhc?get=P1_001N,NAME&for=block:*&in=state:{state_num}&in=county:*&in=tract:*&key=d1588971d5fc38a221e0931f134ad704c62a06e0")
    aund1 = pd.read_json(f"https://api.census.gov/data/2020/dec/dhc?get=P14_003N,P14_028N,NAME&for=block:*&in=state:{state_num}&in=county:*&in=tract:*&key=d1588971d5fc38a221e0931f134ad704c62a06e0")
    a1to4 = pd.read_json(f"https://api.census.gov/data/2020/dec/dhc?get=P14_004N,P14_005N,P14_006N,P14_007N,P14_025N,P14_026N,P14_027N,P14_028N,NAME&for=block:*&in=state:{state_num}&in=county:*&in=tract:*&key=d1588971d5fc38a221e0931f134ad704c62a06e0")
    a5to17 = pd.read_json(f"https://api.census.gov/data/2020/dec/dhc?get=P14_008N,P14_009N,P14_010N,P14_011N,P14_012N,P14_013N,P14_014N,P14_015N,P14_016N,P14_017N,P14_018N,P14_019N,P14_020N,P14_029N,P14_030N,P14_031N,P14_032N,P14_033N,P14_034N,P14_035N,P14_036N,P14_037N,P14_038N,P14_039N,P14_040N,P14_041N,NAME&for=block:*&in=state:{state_num}&in=county:*&in=tract:*&key=d1588971d5fc38a221e0931f134ad704c62a06e0")
    a18to24 = pd.read_json(f"https://api.census.gov/data/2020/dec/dhc?get=P12_007N,P12_008N,P12_009N,P12_010N,P12_031N,P12_032N,P12_033N,P12_034N,NAME&for=block:*&in=state:{state_num}&in=county:*&in=tract:*&key=d1588971d5fc38a221e0931f134ad704c62a06e0")
    aund25 = pd.read_json(f"https://api.census.gov/data/2020/dec/dhc?get=P12_007N,P12_008N,P12_009N,P12_010N,P12_031N,P12_032N,P12_033N,P12_034N,P14_003N,P14_004N,P14_005N,P14_006N,P14_007N,P14_008N,P14_009N,P14_010N,P14_011N,P14_012N,P14_013N,P14_014N,P14_015N,P14_016N,P14_017N,P14_018N,P14_019N,P14_020N,P14_024N,P14_025N,P14_026N,P14_027N,P14_028N,P14_029N,P14_030N,P14_031N,P14_032N,P14_033N,P14_034N,P14_035N,P14_036N,P14_037N,P14_038N,P14_039N,P14_040N,P14_041N,NAME&for=block:*&in=state:{state_num}&in=county:*&in=tract:*&key=d1588971d5fc38a221e0931f134ad704c62a06e0")
    a25ov = pd.read_json(f"https://api.census.gov/data/2020/dec/dhc?get=P12_011N,P12_012N,P12_013N,P12_014N,P12_015N,P12_016N,P12_017N,P12_018N,P12_019N,P12_020N,P12_021N,P12_022N,P12_023N,P12_024N,P12_025N,P12_035N,P12_036N,P12_037N,P12_038N,P12_039N,P12_040N,P12_041N,P12_042N,P12_043N,P12_044N,P12_045N,NAME&for=block:*&in=state:{state_num}&in=county:*&in=tract:*&key=d1588971d5fc38a221e0931f134ad704c62a06e0")
    a25to64 = pd.read_json(f"https://api.census.gov/data/2020/dec/dhc?get=P12_011N,P12_012N,P12_013N,P12_014N,P12_015N,P12_016N,P12_017N,P12_018N,P12_019N,P12_035N,P12_036N,P12_037N,P12_038N,P12_039N,P12_040N,P12_041N,P12_042N,P12_043N,NAME&for=block:*&in=state:{state_num}&in=county:*&in=tract:*&key=d1588971d5fc38a221e0931f134ad704c62a06e0")
    a65to79 = pd.read_json(f"https://api.census.gov/data/2020/dec/dhc?get=P12_020N,P12_021N,P12_022N,P12_023N,P12_035N,P12_036N,P12_037N,P12_038N,P12_039N,P12_040N,P12_041N,P12_042N,P12_043N,NAME&for=block:*&in=state:{state_num}&in=county:*&in=tract:*&key=d1588971d5fc38a221e0931f134ad704c62a06e0")
    a80ov = pd.read_json(f"https://api.census.gov/data/2020/dec/dhc?get=P12_024N,P12_025N,P12_048N,P12_049N,NAME&for=block:*&in=state:{state_num}&in=county:*&in=tract:*&key=d1588971d5fc38a221e0931f134ad704c62a06e0")
    womchild = pd.read_json(f"https://api.census.gov/data/2020/dec/dhc?get=P12_030N,P12_031N,P12_032N,P12_033N,P12_034N,P12_035N,P12_036N,P12_037N,P12_038N,P12_039N,NAME&for=block:*&in=state:{state_num}&in=county:*&in=tract:*&key=d1588971d5fc38a221e0931f134ad704c62a06e0")
    white = pd.read_json(f"https://api.census.gov/data/2020/dec/dhc?get=P3_002N,NAME&for=block:*&in=state:{state_num}&in=county:*&in=tract:*&key=d1588971d5fc38a221e0931f134ad704c62a06e0")
    black = pd.read_json(f"https://api.census.gov/data/2020/dec/dhc?get=P3_003N,NAME&for=block:*&in=state:{state_num}&in=county:*&in=tract:*&key=d1588971d5fc38a221e0931f134ad704c62a06e0")
    amind = pd.read_json(f"https://api.census.gov/data/2020/dec/dhc?get=P3_004N,NAME&for=block:*&in=state:{state_num}&in=county:*&in=tract:*&key=d1588971d5fc38a221e0931f134ad704c62a06e0")
    asian = pd.read_json(f"https://api.census.gov/data/2020/dec/dhc?get=P3_005N,NAME&for=block:*&in=state:{state_num}&in=county:*&in=tract:*&key=d1588971d5fc38a221e0931f134ad704c62a06e0")
    hawpi = pd.read_json(f"https://api.census.gov/data/2020/dec/dhc?get=P3_006N,NAME&for=block:*&in=state:{state_num}&in=county:*&in=tract:*&key=d1588971d5fc38a221e0931f134ad704c62a06e0")
    other = pd.read_json(f"https://api.census.gov/data/2020/dec/dhc?get=P3_007N,NAME&for=block:*&in=state:{state_num}&in=county:*&in=tract:*&key=d1588971d5fc38a221e0931f134ad704c62a06e0")
    twomore = pd.read_json(f"https://api.census.gov/data/2020/dec/dhc?get=P3_008N,NAME&for=block:*&in=state:{state_num}&in=county:*&in=tract:*&key=d1588971d5fc38a221e0931f134ad704c62a06e0")
    nhisp = pd.read_json(f"https://api.census.gov/data/2020/dec/dhc?get=P4_002N,NAME&for=block:*&in=state:{state_num}&in=county:*&in=tract:*&key=d1588971d5fc38a221e0931f134ad704c62a06e0")
    hisp = pd.read_json(f"https://api.census.gov/data/2020/dec/dhc?get=P4_003N,NAME&for=block:*&in=state:{state_num}&in=county:*&in=tract:*&key=d1588971d5fc38a221e0931f134ad704c62a06e0")
    nhispwhite = pd.read_json(f"https://api.census.gov/data/2020/dec/dhc?get=P9_005N,NAME&for=block:*&in=state:{state_num}&in=county:*&in=tract:*&key=d1588971d5fc38a221e0931f134ad704c62a06e0")
    nhispblack = pd.read_json(f"https://api.census.gov/data/2020/dec/dhc?get=P9_006N,NAME&for=block:*&in=state:{state_num}&in=county:*&in=tract:*&key=d1588971d5fc38a221e0931f134ad704c62a06e0")
    household = pd.read_json(f"https://api.census.gov/data/2020/dec/dhc?get=P16_001N,NAME&for=block:*&in=state:{state_num}&in=county:*&in=tract:*&key=d1588971d5fc38a221e0931f134ad704c62a06e0")
    hhpop = pd.read_json(f"https://api.census.gov/data/2020/dec/dhc?get=H8_001N,NAME&for=block:*&in=state:{state_num}&in=county:*&in=tract:*&key=d1588971d5fc38a221e0931f134ad704c62a06e0")
    femhhw18c = pd.read_json(f"https://api.census.gov/data/2020/dec/dhc?get=P12_030N,P12_031N,P12_032N,P12_033N,P12_034N,P12_035N,P12_036N,P12_037N,P12_038N,P12_039N,NAME&for=block:*&in=state:{state_num}&in=county:*&in=tract:*&key=d1588971d5fc38a221e0931f134ad704c62a06e0")
    hhunit = pd.read_json(f"https://api.census.gov/data/2020/dec/dhc?get=H1_001N,NAME&for=block:*&in=state:{state_num}&in=county:*&in=tract:*&key=d1588971d5fc38a221e0931f134ad704c62a06e0")
    occhunit = pd.read_json(f"https://api.census.gov/data/2020/dec/dhc?get=H3_001N,NAME&for=block:*&in=state:{state_num}&in=county:*&in=tract:*&key=d1588971d5fc38a221e0931f134ad704c62a06e0")
    ownerocc = pd.read_json(f"https://api.census.gov/data/2020/dec/dhc?get=H10_001N,NAME&for=block:*&in=state:{state_num}&in=county:*&in=tract:*&key=d1588971d5fc38a221e0931f134ad704c62a06e0")
    vacant = pd.read_json(f"https://api.census.gov/data/2020/dec/dhc?get=H5_001N,NAME&for=block:*&in=state:{state_num}&in=county:*&in=tract:*&key=d1588971d5fc38a221e0931f134ad704c62a06e0")
    hu1p = pd.read_json(f"https://api.census.gov/data/2020/dec/dhc?get=H12_003N,H12_011N,NAME&for=block:*&in=state:{state_num}&in=county:*&in=tract:*&key=d1588971d5fc38a221e0931f134ad704c62a06e0")
    #Using this to reference back the state number
    state = pd.read_json(f"https://api.census.gov/data/2020/dec/dhc?get={variable},NAME&for=state:*&key=d1588971d5fc38a221e0931f134ad704c62a06e0")
    state = state[1:]
    state_name = totalpop[1][1].split(",")[-1]
    print(f"The state you are obtaining data from is{state_name}")
    create_header(totalpop)
    create_header(aund1)
    create_header(a1to4)
    create_header(a5to17)
    create_header(a18to24)
    create_header(aund25)
    create_header(a25ov)
    create_header(a25to64)
    create_header(a65to79)
    create_header(a80ov)
    create_header(womchild)
    create_header(white)
    create_header(black)
    create_header(amind)
    create_header(asian)
    create_header(hawpi)
    create_header(other)
    create_header(twomore)
    create_header(nhisp)
    create_header(hisp)
    create_header(nhispwhite)
    create_header(nhispblack)
    create_header(household)
    create_header(hhpop)
    create_header(femhhw18c)
    create_header(hhunit)
    create_header(occhunit)
    create_header(ownerocc)
    create_header(vacant)
    create_header(hu1p)
    create_sum_col(totalpop, "TOTALPOP")
    create_sum_col(aund1, "AUND1")
    create_sum_col(a1to4, "A1TO4")
    create_sum_col(a5to17, "A5TO17")
    create_sum_col(a18to24, "A18TO24")
    create_sum_col(aund25, "AUND25")
    create_sum_col(a25ov, "A25OV")
    create_sum_col(a25to64, "A25TO64")
    create_sum_col(a65to79, "A65TO79")
    create_sum_col(a80ov, "A80OV")
    create_sum_col(womchild, "WOMCHILD")
    create_sum_col(white, "WHITE")
    create_sum_col(black, "BlACK")
    create_sum_col(amind, "AMIND")
    create_sum_col(asian, "ASIAN")
    create_sum_col(hawpi, "HAWPI")
    create_sum_col(other, "OTHER")
    create_sum_col(twomore, "TWOMORE")
    create_sum_col(nhisp, "NHISP")
    create_sum_col(hisp, "HISP")
    create_sum_col(nhispwhite, "NHISPWHITE")
    create_sum_col(nhispblack, "NHISPBLACK")
    create_sum_col(household, "HOUSEHOLD")
    create_sum_col(hhpop, "HHPOP")
    create_sum_col(femhhw18c, "FEMW18CHILD")
    create_sum_col(hhunit, "HHUNIT")
    create_sum_col(occhunit, "OCC")
    create_sum_col(ownerocc, "OWN")
    create_sum_col(vacant, "VACANT")
    create_sum_col(hu1p, "HU1P")
    merge_df(totalpop, aund1)
    merge_df(totalpop, a1to4)
    merge_df(totalpop, a5to17)
    merge_df(totalpop, a18to24)
    merge_df(totalpop, aund25)
    merge_df(totalpop, a25ov)
    merge_df(totalpop, a25to64)
    merge_df(totalpop, a65to79)
    merge_df(totalpop, a80ov)
    merge_df(totalpop, womchild)
    merge_df(totalpop, white)
    merge_df(totalpop, black)
    merge_df(totalpop, amind)
    merge_df(totalpop, asian)
    merge_df(totalpop, hawpi)
    merge_df(totalpop, other)
    merge_df(totalpop, twomore)
    merge_df(totalpop, nhisp)
    merge_df(totalpop, hisp)
    merge_df(totalpop, nhispwhite)
    merge_df(totalpop, nhispblack)
    merge_df(totalpop, household)
    merge_df(totalpop, hhpop)
    merge_df(totalpop, femhhw18c)
    merge_df(totalpop, hhunit)
    merge_df(totalpop, occhunit)
    merge_df(totalpop, ownerocc)
    merge_df(totalpop, vacant)
    merge_df(totalpop, hu1p)
    totalpop = totalpop.iloc[:,3:]
    totalpop.insert(loc = 0, column = "GEOID", value = totalpop["state"] + totalpop["county"] + totalpop["tract"] + totalpop["block"])
    print(f"Finished processing for{state_name}")
    folder_path = "G:/usgrids2020/processed_census_data"
    os.makedirs(folder_path, exist_ok = True)
    totalpop.to_csv(os.path.join(folder_path,f"{state_name.lower()}_us_final.csv"))
    print(f"Finished exporting for{state_name}")

The state you are obtaining data from is Washington
Finished processing for Washington
Finished exporting for Washington
The state you are obtaining data from is West Virginia
Finished processing for West Virginia
Finished exporting for West Virginia
The state you are obtaining data from is Wisconsin
Finished processing for Wisconsin
Finished exporting for Wisconsin
The state you are obtaining data from is Wyoming
Finished processing for Wyoming
Finished exporting for Wyoming


In [23]:
folder_path = "G:/usgrids2020/processed_census_data/"
gdf_path = "G:/usgrids2020/gdbs/"
gdf_list = os.listdir(gdf_path)
gdf_list.sort()
file_list = os.listdir(folder_path)
file_list.sort()
# for file in file_list:
df = pd.read_csv(os.path.join(folder_path ,file_list[8]))
#     fc = gpd.read_file(os.path.join(gdf_path, f"usa{file[:2]}.gdb"))
df = df.iloc[:,1:]
#     fc1 = fc.sort_values(by="GEOID20")
#     fc1.reset_index(inplace = True)
df1 = df.sort_values(["GEOID"])
df1.reset_index(inplace = True)
#     fc1 = fc1.iloc[:,:16]
#     fc1.columns
new_cols = df1.loc[:,~df1.columns.isin(["index", "GEOID", "state", "county", "tract", "block"])].columns
df
#     fc1[new_cols] = df1.loc[:,~df1.columns.isin(["index", "GEOID", "state", "county", "tract", "block"])]
#     fc1 = fc1.iloc[:,~fc1.columns.isin(["index", "STATEFP20", 'COUNTYFP20','TRACTCE20', 'BLOCKCE20',
#        'NAME20', 'MTFCC20', 'UR20', 'UACE20', 'UATYPE20', 'FUNCSTAT20',
#        'ALAND20', 'AWATER20', 'INTPTLAT20', 'INTPTLON20'])]
output_path = "G:/usgrids2020/for_join_data"
os.makedirs(output_path, exist_ok = True)
# fc1 = fc1.loc[:,~fc1.columns.isin(["NAME20"])]
# fc1['UACE20'] = fc1['UACE20'].str.replace(" ", "")
# fc1['UATYPE20'] = fc1['UATYPE20'].str.replace(" ", "")
# fc1.to_csv(os.path.join(output_path,f"{file[:2]}_inputs.csv"), index = False)
print(f"Finished exporting for {file_list[8][:2]}")


Finished exporting for de


In [ ]:
import arcpy
import os
from multiprocessing import Pool
import datetime

scriptTime = datetime.datetime.now()
def add_join_and_copy(state_gdb):
    try:


        joinTable = os.path.join(folder_path, f"{state_gdb[3:5]}_inputs.csv")
        memory = 'in_memory/'
        mem_table_path = os.path.join(memory, 'csv')

        arcpy.CopyRows_management(joinTable, mem_table_path)

        joined_table = arcpy.management.AddJoin(
            in_layer_or_view=f"usa{state_gdb[3:5]}_admin5_boundaries",
            in_field="GEOID20",
            join_table=joinTable,
            join_field="GEOID20",
            join_type="KEEP_ALL",
            index_join_fields="NO_INDEX_JOIN_FIELDS",
            rebuild_index="NO_REBUILD_INDEX"
        )

        arcpy.management.CopyFeatures(joined_table, f"joinedtable_{state_gdb}")
        print(f"Done for {state_gdb}")
    except Exception as e:
        print(f"Error processing {state_gdb}: {str(e)}")

def main():
# Assuming you have a list of gdb names in 'gdb_list'
    # Modify the following paths accordingly
    gdb_path = r'G:/usgrids2020/gdbs/'
    folder_path = r'G:/usgrids2020/for_join_data/'
    gdb_list=[]
    print("processing")
    for state_gdb in gdb_path:
        arcpy.env.workspace = os.path.join(gdb_path, state_gdb)
        arcpy.env.overwriteOutput = True
        gdb_list.append(state_gdb)
    # Set the number of processes (adjust as needed)
    num_processes = 8
    print("Start pooling")
    with Pool(num_processes) as pool:
        results = pool.map(add_join_and_copy, gdb_list)
    for result in results:
        print(result)
    pool.close
    pool.join()
    print("Script Complete in " + str(datetime.datetime.now() - scriptTime))

if __name__ == '__main__':
    main()


processing
Start pooling


In [70]:
import arcpy
import os
import pandas as pd
gdb_path = "G:/usgrids2020/gdbs/"
gdb_list = os.listdir(gdb_path)
gdb_list.sort()
join_df = pd.read_csv("G:/usgrids2020/for_join_data/ri_inputs.csv")
join_df = join_df.iloc[:,1:]
# for state_gdb in gdb_list:
arcpy.env.workspace = os.path.join(gdb_path, gdb_list[-12])
fc = 'usari_admin5_boundaries'
# for fc in arcpy.ListFeatureClasses():
#     fields = [field.name for field in arcpy.ListFields(fc)]
fields = list(join_df.columns)
values =list()
for field in fields:
    arcpy.management.AddField(fc, field, "LONG")

#     with arcpy.da.UpdateCursor(fc, field) as cursor:
#         for row in cursor:
#             fields = row[0]
#             for key, value in join_df.iterrows():
#                 df_dict = value.to_dict()
#                 values.append(list(df_dict.values()))
#             if files in row[0]:
                